In [2]:
import os
import numpy as np
import DataPreprocessing as DP
from matplotlib import pyplot as plt
from astropy.io import fits
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Input, Flatten
from keras import optimizers
from keras.models import Model



import time
Tstart = time.time()

In [3]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #for GPU usagegpus = tf.config.experimental.list_physical_devices(device_type='XLA_GPU')
gpus = tf.config.experimental.list_physical_devices(device_type='XLA_GPU')
print(gpus)
tf.config.experimental.set_visible_devices(devices=gpus, device_type='XLA_GPU')

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]


In [4]:
#norm spectrum
kinds = ['boss_cv','boss_da+ms','boss_db','boss_db+ms','boss_dq','boss_dz','fgkm','hotstars','wd','wdsb2','yso','hotstars_m']
flux_cv, spectrum_cv = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[0]+'/'+'*.fit')
flux_dams, spectrum_dams = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[1]+'/'+'*.fit')
flux_db, spectrum_db = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[2]+'/'+'*.fit')

#delate bad data
n = 0
for i in range(len(spectrum_db)):
    if len(spectrum_db[i][0]) < 4096:
        n = i
flux_db.pop(n)
spectrum_db.pop(n)

[array([5282.0195, 5283.235 , 5284.453 , 5285.669 , 5286.8877, 5288.104 ,
        5289.323 , 5290.54  , 5291.7573, 5292.9775, 5294.1953, 5295.416 ,
        5296.6343, 5297.8555, 5299.074 , 5300.2935, 5301.5156, 5302.7354,
        5303.958 , 5305.178 , 5306.4014, 5307.622 , 5308.8433, 5310.0674,
        5311.289 , 5312.5137, 5313.736 , 5314.9614, 5316.184 , 5317.407 ,
        5318.6333, 5319.857 , 5321.0835, 5322.3076, 5323.5347, 5324.76  ,
        5325.9844, 5327.213 , 5328.438 , 5329.667 , 5330.893 , 5332.122 ,
        5333.349 , 5334.5757, 5335.806 , 5337.0337, 5338.264 , 5339.492 ,
        5340.7236, 5341.952 , 5343.181 , 5344.413 , 5345.6426, 5346.8755,
        5348.1055, 5349.339 , 5350.5693, 5351.8003, 5353.034 , 5354.2656,
        5355.5005, 5356.7324, 5357.968 , 5359.2   , 5360.433 , 5361.6694,
        5362.903 , 5364.139 , 5365.3735, 5366.611 , 5367.845 , 5369.08  ,
        5370.3184, 5371.5537, 5372.7925, 5374.0283, 5375.2676, 5376.504 ,
        5377.744 , 5378.981 , 5380.218

In [5]:
l_list =  flux_cv+flux_dams+flux_db
sl = []
for i in range(len(l_list)):
    sl.append(len(l_list[i]))
sl_min = np.min(sl)
X_train = []
for i in range(len(l_list)):
    X_train.append(l_list[i][0:sl_min])
size = 4552
X_train = np.array(X_train)

In [6]:
X_train = np.stack(X_train)
X_train = X_train.reshape(1,len(X_train),size)
print(X_train.shape)

(1, 1214, 4552)


In [89]:
# #LISM Net 

# model = Sequential()
# model.add(LSTM(32, input_dim=64, input_length=10, return_sequences=True))
# # note that you only need to specify the input size on the first layer.
# # for subsequent layers, no need to specify the input size:
# model.add(LSTM(16, return_sequences=True))
# model.add(LSTM(10))

# optimizer_adam = optimizers.Adam(lr=0.001)
# model.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])####loss function
# #autoencoder.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 10, 32)            12416     
_________________________________________________________________
lstm_45 (LSTM)               (None, 10, 16)            3136      
_________________________________________________________________
lstm_46 (LSTM)               (None, 10)                1080      
Total params: 16,632
Trainable params: 16,632
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
inputd = Input(shape=(1214,4552))

In [106]:
x = LSTM(128,return_sequences=True)(inputd)
x = LSTM(64,return_sequences=True)(x)
x = LSTM(32)(x)
x = Flatten()(x)
#x = Dense(16,activation='softmax')(x)
x.shape

TensorShape([None, 32])

In [107]:
model = Model(inputd, x)
optimizer_adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])####loss function&accuracy
print(model.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1214, 4552)]      0         
_________________________________________________________________
lstm_62 (LSTM)               (None, 1214, 128)         2396672   
_________________________________________________________________
lstm_63 (LSTM)               (None, 1214, 64)          49408     
_________________________________________________________________
lstm_64 (LSTM)               (None, 32)                12416     
_________________________________________________________________
flatten_3 (Flatten)          (None, 32)                0         
Total params: 2,458,496
Trainable params: 2,458,496
Non-trainable params: 0
_________________________________________________________________
None


In [108]:
Nepochs = 100 #number of epochs for CAE training
tosavemodel = True #if save the trained CAE model
savename = 'LSTM_reconstruction' #setup if "tosavemodel=True" or "plot_reconstuction=True"

In [109]:
Tprocess0 = time.time()
print('\n', '## DATE PREPARATION RUNTIME:', Tprocess0-Tstart) #Timer

## MAIN ##
#training
H = model.fit(X_train, X_train,
                epochs=Nepochs,
                shuffle=True)

Tprocess1 = time.time()
print('\n', '## AE TRAINING RUNTIME:', Tprocess1-Tprocess0) #Timer


 ## DATE PREPARATION RUNTIME: 82.88735723495483
Epoch 1/100


ValueError: in user code:

    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/

In [16]:
X_train.shape

(1214, 4552)

In [119]:
inputs = tf.random.normal([1,1214,4552])
inputs.shape

TensorShape([1, 1214, 4552])

In [122]:
inputs

<tf.Tensor: shape=(1, 1214, 4552), dtype=float32, numpy=
array([[[ 1.5358809 , -0.39456266, -0.31369784, ..., -0.07364149,
          0.9602052 ,  0.9721474 ],
        [ 0.7066232 , -1.1488916 ,  0.6905954 , ...,  0.48022604,
         -1.0171343 , -0.9603228 ],
        [-0.5398689 ,  1.0745703 ,  0.73378175, ..., -0.9619879 ,
          0.28467724,  0.38736975],
        ...,
        [-0.44530717,  1.4420067 , -0.7901288 , ..., -0.45089746,
          1.0791183 ,  0.6114517 ],
        [-0.95790726,  0.93801   , -0.9143517 , ...,  1.3658686 ,
          0.5540797 ,  1.2816627 ],
        [-0.13016294,  0.40100637, -0.8612439 , ...,  0.15350777,
          0.15454826,  1.3567326 ]]], dtype=float32)>

In [120]:
#lstm = tf.keras.layers.LSTM(4)
output1 = tf.keras.layers.LSTM(4)(inputs)

In [121]:
#output = lstm(inputs)
output.shape

TensorShape([1, 4])

In [53]:
inputs = tf.random.normal([1,1,128])

In [ ]:
inputd = Input(shape=4552)

In [54]:
inputd = Input(shape=(1,128))
print(inputd.shape)
x = LSTM(128)(inputd)
# x = LSTM(64)(x)
# x = LSTM(32)(x)
# x = LSTM(16)(x)

(None, 1, 128)


In [55]:
inputs = tf.squeeze(inputs)

In [56]:
model = Model(inputd, x)
optimizer_adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])####loss function&accuracy
print(model.summary())

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 1, 128)]          0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 128)               131584    
Total params: 131,584
Trainable params: 131,584
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
Nepochs = 100 #number of epochs for CAE training
tosavemodel = True #if save the trained CAE model
savename = 'LSTM_reconstruction' #setup if "tosavemodel=True" or "plot_reconstuction=True"

In [58]:
Tprocess0 = time.time()
print('\n', '## DATE PREPARATION RUNTIME:', Tprocess0-Tstart) #Timer

## MAIN ##
#training
H = model.fit(inputs, inputs,
                epochs=Nepochs,
                shuffle=True)

Tprocess1 = time.time()
print('\n', '## AE TRAINING RUNTIME:', Tprocess1-Tprocess0) #Timer


 ## DATE PREPARATION RUNTIME: 1442.3119337558746
Epoch 1/100


ValueError: in user code:

    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__
  